In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from functools import partial

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.ae_templates import conv_architecture_ala_nips_17, default_train_params_ala_nips_17

from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only,\
encoder_with_convs_and_symmetry_new, decoder_with_convs_only

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tflearn.layers.conv import avg_pool_1d
from pc_completions.evaluation import basic_comletion_measures

from tflearn.layers.conv import conv_1d, highway_conv_1d, avg_pool_1d, max_pool_1d
from tf_lab.fundamentals.layers import conv_1d_tranpose

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
exp_id = -1
n_pc_points = 2048
random_seed = 42
top_data_dir = '/orions4-zfs/projects/optas/DATA'
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/testing_ae_settings/ae_variants_2', str(exp_id))

In [5]:
def load_multiple_version_of_pcs():
#     versions = ['centered', 'centered_2nd_version', 'centered_3rd_version']
#     versions = ['fps_sampled_in_u_sphere']    
    versions = ['fps_sampled_in_u_sphere_fiedler_sorted']
    
    splits = {'train':None, 'val':None, 'test': None}
        
    for s in splits.keys():
        print 'Loading %s data.' % (s,)
        s_file = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/', s + '.txt')
        print s_file
        for i, v in enumerate(versions):
            top_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/', v, str(n_pc_points))                
            splitter = Data_Splitter(top_pclouds_path, data_file_ending='.ply', random_seed=random_seed)
            pcs_in_split = splitter.load_splits(s_file)                
            pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(pcs_in_split, n_threads=20, loader=snc_loader, verbose=True)        
            if splits[s] is None:
                splits[s] = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)
            else:
                splits[s].merge(PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids))
    return splits

In [7]:
in_data  = load_multiple_version_of_pcs()

Loading test data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt
679 pclouds were loaded. They belong in 1 shape-classes.
Loading train data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt
5761 pclouds were loaded. They belong in 1 shape-classes.
Loading val data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt
338 pclouds were loaded. They belong in 1 shape-classes.


In [20]:
do_training = True
load_pretrained_model = False
load_epoch = None
loss = 'chamfer'
training_epochs = 500
batch_size = 50
learning_rate = 0.0005

In [25]:
def enc_dec_templates():
    b_norm = True        
    encoder = encoder_with_convs_and_symmetry_new
    decoder = decoder_with_fc_only
    conv_op = conv_1d

    encoder_args = {'n_filters': [32, 64, 128, 256, 32],
            'filter_sizes': [3, 3, 3, 2, 1],
            'strides': [1, 1, 1, 1, 1],
            'b_norm': b_norm,
            'pool': None,
            'pool_sizes': [2, None, 2, None, None],
            'verbose': True,
            'conv_op': conv_op
            }

    decoder_args= {'layer_sizes': [64, 256, 2048*3],
                   'verbose': True,
                   'b_norm': b_norm,
                   'non_linearity': tf.nn.relu
                  }
    return encoder, decoder, encoder_args, decoder_args

In [26]:
reset_tf_graph()
encoder, decoder, encoder_args, decoder_args = enc_dec_templates()
print encoder_args

# layer = tf.placeholder(tf.float32, [None, n_pc_points, 3])
# layer = encoder(layer, **encoder_args)
# layer = decoder(layer, **decoder_args)

{'filter_sizes': [3, 3, 3, 2, 1], 'n_filters': [32, 64, 128, 256, 32], 'verbose': True, 'strides': [1, 1, 1, 1, 1], 'conv_op': <function conv_1d at 0x7f68e3a7c320>, 'pool_sizes': [2, None, 2, None, None], 'b_norm': True, 'pool': None}


In [27]:
if load_pretrained_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    if conf.train_dir != train_dir:
        conf.train_dir = train_dir
#     conf.save(osp.join(train_dir, 'configuration'))
else:    
    conf = Conf(
                n_input = [n_pc_points, 3],
                loss = loss,
                training_epochs = training_epochs,
                batch_size = batch_size,
                denoising = False,
                learning_rate = learning_rate,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = None,
                z_rotate = False,
                encoder = encoder,
                decoder = decoder,
                encoder_args = encoder_args,
                decoder_args = decoder_args
               )

conf.experiment_name = 'experiment_' + str(exp_id)
conf.held_out_step = 5
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)
print ae.trainable_parameters()

Building Encoder
encoder_conv_layer_0 conv params =  320 bnorm params =  64
Tensor("experiment_-1_1/Relu:0", shape=(?, 2048, 32), dtype=float32)
output size: 65536 

encoder_conv_layer_1 conv params =  6208 bnorm params =  128
Tensor("experiment_-1_1/Relu_1:0", shape=(?, 2048, 64), dtype=float32)
output size: 131072 

encoder_conv_layer_2 conv params =  24704 bnorm params =  256
Tensor("experiment_-1_1/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_3 conv params =  65792 bnorm params =  512
Tensor("experiment_-1_1/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
output size: 524288 

encoder_conv_layer_4 conv params =  8224 bnorm params =  64
Tensor("experiment_-1_1/Relu_4:0", shape=(?, 2048, 32), dtype=float32)
output size: 65536 

Tensor("experiment_-1_1/Max:0", shape=(?, 32), dtype=float32)
Building Decoder
decoder_fc_0 FC params =  2112 bnorm params =  128
Tensor("experiment_-1_1/Relu_5:0", shape=(?, 64), dtype=float32)
output size: 64 

dec

In [28]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)     
    train_stats = ae.train(in_data['train'], conf, log_file=fout, held_out_data=in_data['test'])
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.1951', 'loss=', '0.005648473')
('Epoch:', '0002', 'training time (minutes)=', '0.1908', 'loss=', '0.002875799')
('Epoch:', '0003', 'training time (minutes)=', '0.1884', 'loss=', '0.002461173')
('Epoch:', '0004', 'training time (minutes)=', '0.1872', 'loss=', '0.002252355')
('Epoch:', '0005', 'training time (minutes)=', '0.1896', 'loss=', '0.002080258')
('Held Out Data :', 'forward time (minutes)=', '0.0070', 'loss=', '0.001875525')
('Epoch:', '0006', 'training time (minutes)=', '0.1906', 'loss=', '0.001944169')
('Epoch:', '0007', 'training time (minutes)=', '0.1893', 'loss=', '0.001867765')
('Epoch:', '0008', 'training time (minutes)=', '0.1913', 'loss=', '0.001829649')
('Epoch:', '0009', 'training time (minutes)=', '0.1911', 'loss=', '0.001755436')
('Epoch:', '0010', 'training time (minutes)=', '0.1882', 'loss=', '0.001696055')
('Held Out Data :', 'forward time (minutes)=', '0.0063', 'loss=', '0.001594184')
('Epoch:', '0011', 'training

In [24]:
from tf_lab.point_clouds.convenience import reconstruct_pclouds
_, l_test = reconstruct_pclouds(ae, in_data['test'].point_clouds, batch_size=100)
_, l_train = reconstruct_pclouds(ae, in_data['train'].point_clouds, batch_size=100)
print l_test, l_train

0.000749164663748 0.000654067897925


In [ ]:

a = np.arange(16).reshape(1,4,4)
a.repeat(3, axis=2)
a = np.arange(16).reshape(1,4,4)
print a
a = a.reshape(-1)
a = np.tile(a, 3)
# print a
a = np.transpose(a.reshape(1, 3, 4, 4), [0,3,2,1])
print a
print a.shape

n_filters = 5
n_pc_points = 4
# group_size = 20
batch_size = 1

in_layer = tf.placeholder(tf.float32, [None, n_pc_points, 3])

filter_weights = tf.nn.sigmoid(conv_1d(in_layer, group_size, n_filters))

tf.while_loop()


# tf.while_loop


# filter_weights = tf.constant(np.arange(n_pc_points * n_filters, dtype='f4').reshape(batch_size, n_pc_points, n_filters))

# print sess.run(filter_weights)

# filter_weights = tf.reshape(filter_weights, [-1, n_pc_points*n_filters])

# filter_weights = tf.tile(filter_weights, [1,3]) # expand for x-y-z

# filter_weights = tf.transpose(tf.reshape(filter_weights, (1, n_pc_points, n_filters, 3) ), [0, 2, 3, 1])

# sess.run(filter_weights[0][0])